# Validaton of the spherical quadrature equations

In [1]:
import numpy
import numpy.matlib

In [2]:
def rquad(N, k):
    k1 = k+1; k2 = k+2; n = numpy.arange(1, N+1); nnk = 2*n+k
    A = numpy.insert(numpy.matlib.repeat(k**2, N) / (nnk*(nnk+2)),0,k/k2 )
    n = numpy.arange(2, N+1); nnk = nnk[1:N+1]
    B1=4*k1/(k2*k2*(k+3)); nk=n+k; nnk2=nnk*nnk
    B=4*(n*nk)**2/(nnk2*nnk2-nnk2)
    ab = numpy.column_stack((A,numpy.concatenate(([(2**k1)/k1], [B1], B))))
    s = numpy.sqrt(ab[1:N,1])
    [X,V] = numpy.linalg.eig(numpy.diag(ab[0:N,0],0)+numpy.diag(s,-1)+numpy.diag(s,1))
    I = numpy.argsort(X)
    X=numpy.sort(X)
    V = V[:,I]
    x=(X+1)/2
    w=(1/2)**(k1)*ab[0,1]*V[0]**2
    return x, w

In [8]:
def spherequad(nr,nt,np,rad):
    r, wr = rquad(nr,2);         # radial weights and nodes (mapped Jacobi)

    if rad == float('inf'):        # infinite radius sphere
        wr=wr/(1-r)**4           # singular map of sphere radius
        r=r/(1-r) 
    else:                        # finite radius sphere
        wr=wr*rad**3             # Scale sphere radius
        r=r*rad
    x,wt =rquad(nt,0)
    t=numpy.arccos((2*x-1)); wt=2*wt       # theta weights and nodes (mapped Legendre)
    p=2*numpy.pi*numpy.linspace(0, np-1, np)/np         # phi nodes (Gauss-Fourier)
    wp=2*numpy.pi*numpy.ones(np)/np        # phi weights
    rr,tt,pp = numpy.meshgrid(r,t,p)   # Compute the product grid
    
    r = rr.flatten('F')
    t = tt.flatten('F')
    p = pp.flatten('F')

    wt = wt[:, numpy.newaxis]
    wr = wr[:, numpy.newaxis]
    wp = wp[:, numpy.newaxis]
    w=numpy.reshape(
        numpy.dot(numpy.reshape(numpy.dot(wt,wr.transpose()),(nr*nt,1),'F'),
        wp.transpose()),(nr*nt*np,1),'F')  
    w = w.reshape(-1)
    return r, t, p, w

In [9]:
r,t,p,w = spherequad(4,5,6,2)

In [10]:
t

array([2.7049577 , 2.13941585, 1.57079633, 1.0021768 , 0.43663495,
       2.7049577 , 2.13941585, 1.57079633, 1.0021768 , 0.43663495,
       2.7049577 , 2.13941585, 1.57079633, 1.0021768 , 0.43663495,
       2.7049577 , 2.13941585, 1.57079633, 1.0021768 , 0.43663495,
       2.7049577 , 2.13941585, 1.57079633, 1.0021768 , 0.43663495,
       2.7049577 , 2.13941585, 1.57079633, 1.0021768 , 0.43663495,
       2.7049577 , 2.13941585, 1.57079633, 1.0021768 , 0.43663495,
       2.7049577 , 2.13941585, 1.57079633, 1.0021768 , 0.43663495,
       2.7049577 , 2.13941585, 1.57079633, 1.0021768 , 0.43663495,
       2.7049577 , 2.13941585, 1.57079633, 1.0021768 , 0.43663495,
       2.7049577 , 2.13941585, 1.57079633, 1.0021768 , 0.43663495,
       2.7049577 , 2.13941585, 1.57079633, 1.0021768 , 0.43663495,
       2.7049577 , 2.13941585, 1.57079633, 1.0021768 , 0.43663495,
       2.7049577 , 2.13941585, 1.57079633, 1.0021768 , 0.43663495,
       2.7049577 , 2.13941585, 1.57079633, 1.0021768 , 0.43663

In [11]:
p

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.04719755, 1.04719755, 1.04719755, 1.04719755, 1.04719755,
       1.04719755, 1.04719755, 1.04719755, 1.04719755, 1.04719755,
       1.04719755, 1.04719755, 1.04719755, 1.04719755, 1.04719755,
       1.04719755, 1.04719755, 1.04719755, 1.04719755, 1.04719755,
       2.0943951 , 2.0943951 , 2.0943951 , 2.0943951 , 2.0943951 ,
       2.0943951 , 2.0943951 , 2.0943951 , 2.0943951 , 2.0943951 ,
       2.0943951 , 2.0943951 , 2.0943951 , 2.0943951 , 2.0943951 ,
       2.0943951 , 2.0943951 , 2.0943951 , 2.0943951 , 2.0943951 ,
       3.14159265, 3.14159265, 3.14159265, 3.14159265, 3.14159265,
       3.14159265, 3.14159265, 3.14159265, 3.14159265, 3.14159265,
       3.14159265, 3.14159265, 3.14159265, 3.14159265, 3.14159

In [12]:
w

array([0.02054789, 0.0415099 , 0.04933787, 0.0415099 , 0.02054789,
       0.13622962, 0.27520474, 0.32710309, 0.27520474, 0.13622962,
       0.28474763, 0.57523391, 0.68371203, 0.57523391, 0.28474763,
       0.22009954, 0.44463484, 0.52848448, 0.44463484, 0.22009954,
       0.02054789, 0.0415099 , 0.04933787, 0.0415099 , 0.02054789,
       0.13622962, 0.27520474, 0.32710309, 0.27520474, 0.13622962,
       0.28474763, 0.57523391, 0.68371203, 0.57523391, 0.28474763,
       0.22009954, 0.44463484, 0.52848448, 0.44463484, 0.22009954,
       0.02054789, 0.0415099 , 0.04933787, 0.0415099 , 0.02054789,
       0.13622962, 0.27520474, 0.32710309, 0.27520474, 0.13622962,
       0.28474763, 0.57523391, 0.68371203, 0.57523391, 0.28474763,
       0.22009954, 0.44463484, 0.52848448, 0.44463484, 0.22009954,
       0.02054789, 0.0415099 , 0.04933787, 0.0415099 , 0.02054789,
       0.13622962, 0.27520474, 0.32710309, 0.27520474, 0.13622962,
       0.28474763, 0.57523391, 0.68371203, 0.57523391, 0.28474